In [1]:
#import packages and define functions

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from statsmodels import robust
from collections import OrderedDict, defaultdict
import warnings
from functools import reduce

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

#define index of dispersion function as standard deviation squared over the mean intensity
def index_of_dispersion(df, stddev_column,mean_intensity_column):
    stddev_squared = df[stddev_column].apply(lambda x: x*x)
    d = stddev_squared/df[mean_intensity_column]
    return d


#parameters for marker perturbation 
def marker_perturbation_score(df):
    if (-2.5 < df['mad_z_score'] < 2.5):
        return 0
    elif (-5 < df['mad_z_score'] < -2.5):
        return 0.5
    elif (2.5 < df['mad_z_score'] < 5):
        return 0.5
    elif (df['mad_z_score'] < -5):
        return 1
    elif (df['mad_z_score'] > 5):
        return 1


    
#function to take in dataframe and return mad_z_score    
def score(df,ids, groups,raw_value, ad_value):
    df1 = df.melt(id_vars = ids, value_vars = [raw_value, ad_value],var_name = ['inputs'])
    
    dmso_only= df1[df1.compound_name == 'DMSO'] #selects for control values
    
    #calculate control medians
    control_median = dmso_only.loc[dmso_only['inputs'] == raw_value].groupby(groups).agg(
        'median').reset_index().rename(columns = {'value': 'control_medians'}) 
    
   #calculate control mads by taking the median of the absolute deviation 
    control_mad = dmso_only[dmso_only.inputs == ad_value].groupby(groups).agg(
        'median').reset_index().rename(columns = {'value': 'control_mad'})
    
    #merge median and mad columns
    medians_and_mads = control_median.merge(control_mad)
    
    #merge control medians and mads back into original data frame
    melted_harmony_file_by_control = df1.merge(medians_and_mads, how = 'inner' ,on = ['channel488','channel568','channel647','cell_line','user_timepoint'])    
    
    #remove absolute deviation values as they are not needed
    harmony_file_by_control = melted_harmony_file_by_control[melted_harmony_file_by_control.inputs != ad_value]
    
    # calculate mad_z_score by subtracting a given CV or D value from its control median and divide by the mad 
    harmony_file_by_control.insert(harmony_file_by_control.shape[1], 'mad_z_score', (
        (harmony_file_by_control.loc[:,'value']) - (harmony_file_by_control.loc[:,'control_medians']))/(harmony_file_by_control.loc[:,'control_mad']))
    #clean
    mad_z_score = harmony_file_by_control.copy().drop(columns = ['compound_conc_y'])   
    #apply marker perturbation function to each value of mad_z_score
    mad_z_score['marker_perturbation_score'] = mad_z_score.apply(marker_perturbation_score, axis = 1)
    
    mad_z_score
    
    return mad_z_score 



Import files

In [2]:

#copy and paste path name over the current red pathname 
harmony_file_1 = pd.read_csv('/Users/juliannalamm/Library/CloudStorage/OneDrive-SharedLibraries-DewpointTherapeutics/dpaint - dpaint_data/core_experiments/dpaint-007_eDM-015_HeLa_DT480/harmony_evaluations/20220511_d.paint-007_DM1_no_duplicates.csv') #import files
# drops the notes header in original file, not needed if there is no 
# harmony_file_1 = raw_harmony_file_1.rename(columns = raw_harmony_file_1.iloc[0]).drop(raw_harmony_file_1.index[0])

#convert column and row numbers to string for processing later 
harmony_file_1['column'] = harmony_file_1['column'].astype(str)
harmony_file_1['row'] = harmony_file_1['row'].astype(str)


Calculate Index of Dispersion (D) 

In [4]:


# separate columns by channel, can delete channels that are not needed 
stddev488 = ['nucleus_stddev_mean_perwell_488','cytoplasm_stddev_mean_perwell_488']

stddev568 = ['nucleus_stddev_mean_perwell_568','cytoplasm_stddev_mean_perwell_568']

stddev647 = ['nucleus_stddev_mean_perwell_647','cytoplasm_stddev_mean_perwell_647']

mean_intensity488 = ['nucleus_intensity_mean_perwell_488','cytoplasm_intensity_mean_perwell_488']

mean_intensity568 = ['nucleus_intensity_mean_perwell_568','cytoplasm_intensity_mean_perwell_568']
mean_intensity647 = ['nucleus_intensity_mean_perwell_647','cytoplasm_intensity_mean_perwell_647']


#columns from original dataframe to be used in d calculations
stddev_columns = stddev488 + stddev568 + stddev647
mean_intensity_columns = mean_intensity488 + mean_intensity568 + mean_intensity647

# iterate through column values and create new table with d values 
d_values = []
for (i,j) in zip(stddev_columns, mean_intensity_columns):
    d = index_of_dispersion(harmony_file_1,i,j)
    d_values.append(d)
d_column_names = ['cyto_d_488','nuc_d_488','nuc_d_568','cyto_d_568','nuc_d_647','cyto_d_647']
d_values_table = pd.DataFrame(d_values).transpose()
d_values_table.columns = d_column_names

#add new Index of dispersion Columns to original table
harmony_file_with_d = pd.concat([d_values_table, harmony_file_1],axis=1)
harmony_file_with_d

,cyto_d_488,nuc_d_488,nuc_d_568,cyto_d_568,nuc_d_647,cyto_d_647,Unnamed: 0,plate_id,well_position,row,...,cell_compartment,user_timepoint,channel405,channel405_order,channel488,channel488_order,channel568,channel568_order,channel647,channel647_order
0,8671.688740,224.071836,77.602996,14.457650,140.134196,2.240154,0,103509,C18,3,...,Nucleus,18h,Hoescht/CMB,2.0,COIL,1.0,NucleolusRed,3.0,TCAB1/WRAP53,4.0
1,1388.866975,153.649526,4.000782,12.765820,1995.528697,128.714476,2,103509,C19,3,...,Nucleus,18h,Hoescht/CMB,2.0,SRSF2,1.0,NucleolusRed,3.0,SON,4.0
2,6310.873247,201.287510,95.005993,31.232548,146.078877,3.149305,4,103509,C5,3,...,Nucleus,18h,Hoescht/CMB,2.0,COIL,1.0,NucleolusRed,3.0,TCAB1/WRAP53,4.0
3,144.267522,91.669910,51.710948,8.829919,24.478128,18.204623,6,103509,D5,4,...,Cytoplasm,18h,Hoescht/CMB,2.0,PCNT,1.0,NucleolusRed,3.0,PCM1,4.0
4,263.116230,187.457411,78.216032,24.940934,370.972379,661.932170,8,103509,D8,4,...,Cytoplasm,18h,Hoescht/CMB,2.0,hnRNP,1.0,NucleolusRed,3.0,G3BP1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.589760,0.741187,76.179561,28.088303,0.137086,0.153788,422,103509,N11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,0.603575,0.745075,71.532871,15.032808,0.161501,0.186095,424,103509,N14,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,0.645799,0.734179,71.335172,11.552210,0.167754,0.178543,426,103509,N18,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,0.685462,0.763042,66.824194,12.916414,0.175093,0.174539,428,103509,N19,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# generate medians for CV and d
channels = ('488','568','647')

for channels in channels: 
    harmony_file_with_d['median_cyto_cv_' + str(channels)] = (harmony_file_with_d.groupby(['compound_name','compound_conc','channel'+ str(channels),'cell_line','user_timepoint'])['cyto_cv_'+ str(channels)].transform(np.median))
    harmony_file_with_d['median_cyto_d_' + str(channels)] = (harmony_file_with_d.groupby(['compound_name','compound_conc','channel'+ str(channels),'cell_line','user_timepoint'])['cyto_d_'+ str(channels)].transform(np.median))
    harmony_file_with_d['median_nuc_cv_' + str(channels)] = (harmony_file_with_d.groupby(['compound_name','compound_conc','channel'+ str(channels),'cell_line','user_timepoint'])['nuc_cv_'+ str(channels)].transform(np.median))
    harmony_file_with_d['median_nuc_d_' + str(channels)] = (harmony_file_with_d.groupby(['compound_name','compound_conc','channel'+ str(channels),'cell_line','user_timepoint'])['nuc_d_'+ str(channels)].transform(np.median))
harmony_file_with_medians = harmony_file_with_d.set_index(['compound_name','channel568','channel488','channel647','cell_line','compound_conc','user_timepoint','row','column'])

channels = ('488','568','647')
# get the absolute deviations 
for channels in channels: 
    harmony_file_with_medians['ad_cyto_cv_' + str(channels)] =np.abs((harmony_file_with_medians['cyto_cv_' + str(channels)]).sub(harmony_file_with_medians['median_cyto_cv_' + str(channels)]))
    harmony_file_with_medians['ad_nuc_cv_' + str(channels)] = np.abs((harmony_file_with_medians['nuc_cv_' + str(channels)]).sub(harmony_file_with_medians['median_nuc_cv_' + str(channels)]))
    harmony_file_with_medians['ad_cyto_d_' + str(channels)] = np.abs((harmony_file_with_medians['cyto_d_' + str(channels)]).sub(harmony_file_with_medians['median_cyto_d_' + str(channels)]))
    harmony_file_with_medians['ad_nuc_d_' + str(channels)] = np.abs((harmony_file_with_medians['nuc_d_' + str(channels)]).sub(harmony_file_with_medians['median_cyto_d_' + str(channels)]))


In [6]:
# all_values = ['cyto_cv_488','cyto_cv_647','ad_cyto_cv_488','ad_cyto_cv_647','nuc_cv_488', 'nuc_cv_647','ad_nuc_cv_488','ad_nuc_cv_647', 'cyto_d_488',
#          'cyto_d_647', 'ad_cyto_d_488','ad_cyto_d_647','nuc_d_488','nuc_d_647','ad_nuc_d_488','ad_nuc_d_647']

ad_values =  ['ad_cyto_cv_488','ad_cyto_cv_568','ad_cyto_cv_647','ad_nuc_cv_488','ad_nuc_cv_568','ad_nuc_cv_647', 'ad_cyto_d_488','ad_cyto_d_568','ad_cyto_d_647','ad_nuc_d_488','ad_nuc_d_568','ad_nuc_d_647']

raw_values = ['cyto_cv_488','cyto_cv_568','cyto_cv_647','nuc_cv_488', 'nuc_cv_568', 'nuc_cv_647', 'cyto_d_488', 'cyto_d_568','cyto_d_647', 'nuc_d_488','nuc_d_568','nuc_d_647']

groups = ['channel488','channel568','channel647','cell_line','user_timepoint','compound_conc']

ids = ['channel488','channel568','channel647','compound_name','cell_line','user_timepoint','compound_conc','condensate_name','cell_compartment','row','column','cell_count']

df = harmony_file_with_medians.reset_index()
df


,compound_name,channel568,channel488,channel647,cell_line,compound_conc,user_timepoint,row,column,cyto_d_488,...,ad_cyto_d_488,ad_nuc_d_488,ad_cyto_cv_568,ad_nuc_cv_568,ad_cyto_d_568,ad_nuc_d_568,ad_cyto_cv_647,ad_nuc_cv_647,ad_cyto_d_647,ad_nuc_d_647
0,DMSO,NucleolusRed,COIL,TCAB1/WRAP53,HeLa DT480,0.0,18h,3,18,8671.688740,...,1162.512368,7285.104537,3.674135,5.994456,2.584736,65.730082,0.542129,0.289100,0.223853,138.117894
1,DMSO,NucleolusRed,SRSF2,SON,HeLa DT480,0.0,18h,3,19,1388.866975,...,1393.112238,2628.329686,0.188842,29.500721,0.892906,7.872133,2.416169,1.891684,40.371578,1826.442642
2,DMSO,NucleolusRed,COIL,TCAB1/WRAP53,HeLa,0.0,18h,3,5,6310.873247,...,448.839803,5660.745935,4.385622,4.260291,9.793708,73.567153,0.320284,1.892582,0.147880,143.077452
3,DMSO,NucleolusRed,PCNT,PCM1,HeLa DT480,0.0,18h,4,5,144.267522,...,83.977506,136.575118,2.756162,3.631697,3.042995,39.838033,3.990536,2.585809,5.680504,0.593000
4,DMSO,NucleolusRed,hnRNP,G3BP1,HeLa,0.0,18h,4,8,263.116230,...,4.886260,80.545078,2.574193,2.199862,3.502094,56.777193,0.687451,2.139602,4.808530,286.151261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,NaN,NaN,NaN,NaN,HeLa,NaN,NaN,14,11,0.589760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,NaN,NaN,NaN,NaN,HeLa,NaN,NaN,14,14,0.603575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,NaN,NaN,NaN,NaN,HeLa DT480,NaN,NaN,14,18,0.645799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,NaN,NaN,NaN,NaN,HeLa,NaN,NaN,14,19,0.685462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
d = {}
for (raw_value, ad_value) in zip(raw_values, ad_values):
    d[raw_value] = score(df,ids, groups,raw_value, ad_value)
    if 'nuc' in raw_value:
        d[raw_value] = d[raw_value].loc[d[raw_value]['cell_compartment'] == 'Nucleus']
    if 'cyto' in raw_value:
        d[raw_value] = d[raw_value].loc[d[raw_value]['cell_compartment'] == 'Cytoplasm']
    if '488' in raw_value:
        d[raw_value] = d[raw_value].drop(columns = ['channel647','channel568']).rename(columns = {'channel488':'marker'})
    if '647' in raw_value:
        d[raw_value] = d[raw_value].drop(columns = ['channel488','channel568']).rename(columns = {'channel647':'marker'})
    if '568' in raw_value: 
        d[raw_value] = d[raw_value].drop(columns = ['channel647','channel488']).rename(columns = {'channel568':'marker'})
    if 'd' in raw_value:
        d[raw_value] = d[raw_value].rename(columns = {'value': 'iod','mad_z_score': 'iod_mad_z_score'})
    if 'cv' in raw_value:
        d[raw_value] = d[raw_value].rename(columns = {'value': 'cv','mad_z_score': 'cv_mad_z_score'})

        
data = {}
for key, dataframe in d.items():
    dataframe['median_score'] = dataframe.groupby(['compound_name','cell_line','compound_conc_x','user_timepoint','marker'])['marker_perturbation_score'].transform('median') 
#     dataframe['Summed_median_Score'] = dataframe.groupby(['compound_id','cell_line','compound_conc_x','user_timepoint'])['median_Score'].transform('median')
#     dataframe['Global_perturbation_score'] =dataframe['Summed_Score']/(len(np.unique(dataframe['marker'])))
    data[key] = dataframe.drop(columns = ['inputs','control_medians','control_mad'])

d_dataframes = [] 
cv_dataframes = []
for key in data:
    if 'd' in key:
        d_dataframes.append(data[key])
    if 'cv' in key:
        cv_dataframes.append(data[key]) 
concat_d = pd.concat(d_dataframes).rename(columns = {'marker_perturbation_score':'d_marker_perturbation_score', 'median_score':'d_median_marker_perturbation_score'}).set_index(
    ['compound_name','marker','cell_line','user_timepoint','compound_conc_x','condensate_name','cell_compartment','row','column','cell_count_x'])
concat_cv = pd.concat(cv_dataframes).rename(columns = {'marker_perturbation_score':'cv_marker_perturbation_score', 'median_score':'cv_median_marker_perturbation_score'}).set_index(
    ['compound_name','marker','cell_line','user_timepoint','compound_conc_x','condensate_name','cell_compartment','row','column','cell_count_x'])
concat_cv

cv  \
compound_name marker     cell_line  user_timepoint compound_conc_x condensate_name cell_compartment row column cell_count_x               
DMSO          PCNT       HeLa DT480 18h            0.0             Centrosome      Cytoplasm        4   5      16             62.478429   
                                                                                                    9   22     54             78.875901   
Digoxin       PCNT       HeLa DT480 18h            0.2             Centrosome      Cytoplasm        3   16     145           106.770202   
                                                                                                        20     154            77.082745   
                                                                                                        6      357            82.634436   
...                                                                                                                                 ...   
              POLR2A CTD HeLa       18h            0.2             Transcriptional Nucleus          14  9      21             39.852032   
                                                                                                    3   11     265            40.521147   
                                                   4.0             Transcriptional Nucleus          3   21     141            30.413444   
                                                                                                    5   10     106            31.650924   
                                                                                                    7   19     131            32.427261   

                                                                                                                             cell_count_y  \
compound_name marker     cell_line  user_timepoint compound_conc_x condensate_name cell_compartment row column cell_count_x                 
DMSO          PCNT       HeLa DT480 18h            0.0             Centrosome      Cytoplasm        4   5      16                    35.0   
                                                                                                    9   22     54                    35.0   
Digoxin       PCNT       HeLa DT480 18h            0.2             Centrosome      Cytoplasm        3   16     145                   35.0   
                                                                                                        20     154                   35.0   
                                                                                                        6      357                   35.0   
...                                                                                                                                   ...   
              POLR2A CTD HeLa       18h            0.2             Transcriptional Nucleus          14  9      21                    14.0   
                                                                                                    3   11     265                   14.0   
                                                   4.0             Transcriptional Nucleus          3   21     141                   14.0   
                                                                                                    5   10     106                   14.0   
                                                                                                    7   19     131                   14.0   

                                                                                                                             cv_mad_z_score  \
compound_name marker     cell_line  user_timepoint compound_conc_x condensate_name cell_compartment row column cell_count_x                   
DMSO          PCNT       HeLa DT480 18h            0.0             Centrosome      Cytoplasm        4   5      16                 -1.000000   
                                                              

In [8]:
marker_perturbation= pd.concat([concat_cv,concat_d], axis = 1).reset_index().drop(columns = ['cell_count_y'])
marker_perturbation['cv_summed_scores'] = marker_perturbation.groupby(['compound_name','cell_line','compound_conc_x','user_timepoint'])['cv_median_marker_perturbation_score'].transform('sum') 
marker_perturbation['d_summed_scores'] = marker_perturbation.groupby(['compound_name','cell_line','compound_conc_x','user_timepoint'])['d_median_marker_perturbation_score'].transform('sum') 
marker_perturbation['number_of_replicates'] = marker_perturbation.groupby(['compound_name','cell_line','compound_conc_x','user_timepoint','marker'])['d_summed_scores'].transform('count')
marker_perturbation['d_global_perturbation_score'] = marker_perturbation['d_summed_scores']/(len(np.unique(marker_perturbation['marker'])))/marker_perturbation['number_of_replicates'] #divide by the number of replicates 
marker_perturbation['cv_global_perturbation_score'] = marker_perturbation['cv_summed_scores']/(len(np.unique(marker_perturbation['marker'])))/marker_perturbation['number_of_replicates']
marker_perturbation

# marker_perturbation.to_excel('/Users/juliannalamm/Library/CloudStorage/OneDrive-SharedLibraries-DewpointTherapeutics/dpaint - dpaint_data/core_experiments/dpaint-007_eDM-015_HeLa_DT480/data_analysis/20220511_d.paint-007_MAD_Z.xlsx')